In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pickle
import os 
import numpy as np
import getpass
import os


input_test_file = "/home/catsudon/nvidia/battery_timeserie/resources/raw/test_bat_data.pkl"

with open(input_test_file, "rb") as file:
    test_dict = pickle.load(file)
    
for battery_id in test_dict:
    trimmed_arr = np.trim_zeros(test_dict[battery_id]['q_d_n'], 'b')
    test_dict[battery_id]['trimmed_q_d_n'] = list(trimmed_arr)
    
last_k_th_cycles_list = [10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# calculate slope from first cycle to the last cycle
for battery_id in test_dict:
    q_d_n_values  = test_dict[battery_id]['trimmed_q_d_n']
    cycle = test_dict[battery_id]['cycle']
    slope = (q_d_n_values[-1] - q_d_n_values[0]) / cycle
    test_dict[battery_id]['slope_all_cycles'] = slope
    
for k in last_k_th_cycles_list:
    for battery_id in test_dict:
        q_d_n_values  = test_dict[battery_id]['trimmed_q_d_n']
        cycle = test_dict[battery_id]['cycle']
        if cycle > k:
            slope = (q_d_n_values[-1] - q_d_n_values[-k]) / k
        else:
            slope = np.nan
        test_dict[battery_id][f'slope_last_{k}_cycles'] = slope
        
#calculate a mean gradient of the last k cycles using numpy gradient
for k in last_k_th_cycles_list:
    for battery_id in test_dict:
        q_d_n_values  = test_dict[battery_id]['trimmed_q_d_n']
        cycle = test_dict[battery_id]['cycle']
        if cycle > k:
            slope = np.gradient(q_d_n_values[-k:], 1)
            mean_slope = np.mean(slope)
        else:
            mean_slope = np.nan
        test_dict[battery_id][f'mean_grad_last_{k}_cycles'] = mean_slope

In [4]:
sample_battery_id = 'b3c45'
print("battery_id:", test_dict.keys())
features = test_dict[sample_battery_id].keys()
print(f"sample_battery_id ({sample_battery_id}) features:")
for feature in features:
    print(f'    -{feature} : ({type(test_dict[sample_battery_id][feature])})')

battery_id: dict_keys(['b3c0', 'b3c1', 'b3c3', 'b3c4', 'b3c5', 'b3c6', 'b3c7', 'b3c8', 'b3c9', 'b3c10', 'b3c11', 'b3c12', 'b3c13', 'b3c14', 'b3c15', 'b3c16', 'b3c17', 'b3c18', 'b3c19', 'b3c20', 'b3c21', 'b3c22', 'b3c24', 'b3c25', 'b3c26', 'b3c27', 'b3c28', 'b3c29', 'b3c30', 'b3c31', 'b3c33', 'b3c34', 'b3c35', 'b3c36', 'b3c40', 'b3c41', 'b3c42', 'b3c43', 'b3c44', 'b3c45'])
sample_battery_id (b3c45) features:
    -q_d_n : (<class 'list'>)
    -cycle : (<class 'int'>)
    -trimmed_q_d_n : (<class 'list'>)
    -slope_all_cycles : (<class 'float'>)
    -slope_last_10_cycles : (<class 'float'>)
    -slope_last_50_cycles : (<class 'float'>)
    -slope_last_100_cycles : (<class 'float'>)
    -slope_last_200_cycles : (<class 'float'>)
    -slope_last_300_cycles : (<class 'float'>)
    -slope_last_400_cycles : (<class 'float'>)
    -slope_last_500_cycles : (<class 'float'>)
    -slope_last_600_cycles : (<class 'float'>)
    -slope_last_700_cycles : (<class 'float'>)
    -slope_last_800_cycles : 

In [5]:
os.environ["NEO4J_URI"] = "neo4j+s://3b31837b.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "D4W3Zfi44nAJfStBuxSE2DpKhlk_nMP6ybEjvOX5qxw"

from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(refresh_schema=False)

graph.refresh_schema()
print(graph.schema)

/tmp/ipykernel_4310/636819369.py:6: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(refresh_schema=False)


Node properties:
ChargingPolicy {charging_policy: STRING}
Battery {battery_id: STRING, total_cycles: INTEGER, slope_all_cycles: FLOAT, trimmed_q_d_n_avg: FLOAT, name: STRING, slope_last_10_cycles: FLOAT, slope_last_50_cycles: FLOAT, slope_last_100_cycles: FLOAT, slope_last_200_cycles: FLOAT, slope_last_300_cycles: FLOAT, slope_last_400_cycles: FLOAT, slope_last_500_cycles: FLOAT, slope_last_600_cycles: FLOAT, slope_last_700_cycles: FLOAT, slope_last_800_cycles: FLOAT, slope_last_900_cycles: FLOAT, slope_last_1000_cycles: FLOAT, mean_grad_last_10_cycles: FLOAT, mean_grad_last_50_cycles: FLOAT, mean_grad_last_100_cycles: FLOAT, mean_grad_last_200_cycles: FLOAT, mean_grad_last_300_cycles: FLOAT, mean_grad_last_400_cycles: FLOAT, mean_grad_last_500_cycles: FLOAT, mean_grad_last_600_cycles: FLOAT, mean_grad_last_700_cycles: FLOAT, mean_grad_last_800_cycles: FLOAT, mean_grad_last_900_cycles: FLOAT, mean_grad_last_1000_cycles: FLOAT}
Relationship properties:

The relationships:
(:ChargingPoli

## Example Query

In [19]:
from pprint import pprint
pprint(test_dict['b3c3'])

{'cycle': 1116,
 'mean_grad_last_1000_cycles': -0.00018278807401657106,
 'mean_grad_last_100_cycles': -0.0007513692975044251,
 'mean_grad_last_10_cycles': -0.0005299955606460572,
 'mean_grad_last_200_cycles': -0.0006169727444648742,
 'mean_grad_last_300_cycles': -0.0004754801591237386,
 'mean_grad_last_400_cycles': -0.0003807681798934936,
 'mean_grad_last_500_cycles': -0.000319430947303772,
 'mean_grad_last_50_cycles': -0.0007365322113037109,
 'mean_grad_last_600_cycles': -0.0002763838569323222,
 'mean_grad_last_700_cycles': -0.00024350694247654507,
 'mean_grad_last_800_cycles': -0.00021823398768901824,
 'mean_grad_last_900_cycles': -0.00019931296507517496,
 'q_d_n': [1.062273621559143,
           1.0628892183303833,
           1.0632294416427612,
           1.0635850429534912,
           1.064354658126831,
           1.0645473003387451,
           1.0647578239440918,
           1.0649096965789795,
           1.0648976564407349,
           1.0651054382324219,
           1.0649564266204

In [16]:
# ['b3c0', 'b3c1', 'b3c3', 'b3c4', 'b3c5', 'b3c6', 'b3c7', 'b3c8', 'b3c9', 'b3c10', 'b3c11', 'b3c12', 'b3c13', 'b3c14', 'b3c15', 'b3c16', 'b3c17', 'b3c18', 'b3c19', 'b3c20', 'b3c21', 'b3c22', 'b3c24', 'b3c25', 'b3c26', 'b3c27', 'b3c28', 'b3c29', 'b3c30', 'b3c31', 'b3c33', 'b3c34', 'b3c35', 'b3c36', 'b3c40', 'b3c41', 'b3c42', 'b3c43', 'b3c44', 'b3c45']
test_key = 'b3c3'
test_battery = test_dict[test_key]


# Define feature for similarity comparison
feature = "slope_last_500_cycles"
test_slope = test_battery[f"{feature}"]  # Change this to compare different features
print(test_slope)

# Define similarity threshold and number of results
THRESHOLD = 10  # Adjust based on desired similarity
TOP_K = 3  # Number of closest matches to return

# -------- SINGLE QUERY: Find closest batteries & their charging policies -------- #
query = f"""
MATCH (cp:ChargingPolicy)-[:USED_BY]->(b:Battery)
WHERE abs(b.{feature} - $test_slope) < $threshold
RETURN b.battery_id AS battery_id, 
       b.{feature} AS {feature}, 
       abs(b.{feature} - $test_slope) AS similarity,
       cp.charging_policy AS charging_policy
ORDER BY similarity ASC
LIMIT $top_k
"""

# Execute query with parameters
results = graph.query(query, params={"test_slope": test_slope, "threshold": THRESHOLD, "top_k": TOP_K})

# -------- Display Results -------- #
print(f"Test Battery: {test_key} ({feature} = {test_slope})")
print(f"Closest {TOP_K} Matches in Neo4j (within threshold {THRESHOLD}):")

for idx, battery in enumerate(results):
    battery_id = battery["battery_id"]
    battery_feature_value = battery[f"{feature}"]
    similarity = battery["similarity"]
    charging_policy = battery["charging_policy"] if battery["charging_policy"] else "Unknown"

    print(f"{idx+1}. Battery: {battery_id} - {feature}: {battery_feature_value} (Diff: {similarity})")
    print(f"   Charging Policy: {charging_policy}\n")


-0.00031944239139556886
Test Battery: b3c3 (slope_last_500_cycles = -0.00031944239139556886)
Closest 3 Matches in Neo4j (within threshold 10):
1. Battery: b1c3 - slope_last_500_cycles: -0.0003280808925628662 (Diff: 8.638501167297326e-06)
   Charging Policy: 4C(80%)-4C

2. Battery: b1c11 - slope_last_500_cycles: -0.00034225058555603026 (Diff: 2.2808194160461393e-05)
   Charging Policy: 5.4C(50%)-3C

3. Battery: b1c18 - slope_last_500_cycles: -0.00034250414371490476 (Diff: 2.30617523193359e-05)
   Charging Policy: 5.4C(70%)-3C

